# Case Study of Whitelee Windfarm
This file is for exploring the case study. It is originally placed at the /root folder.

## Land cover types at current wind turbine locations

In [1]:
import numpy as np
import netCDF4
from Land.main import land
from CRS.crs_init import CRSConvertor
from Optimiser.config_ss import cell_width

In [2]:
# read wind turbine locations
turbine_loc = np.loadtxt('Aborted/case/Turbines_At_Whitelee_Wind_Farm.csv', delimiter=',', encoding='utf-8', usecols=[1, 2])
conv = CRSConvertor([55.714704134580245, -4.364543821199962, 55.634359319706036, -4.183104393719847], cell_width)
print(turbine_loc)

[[55.6891561 -4.3580354]
 [55.6866291 -4.3518556]
 [55.6952387 -4.3633451]
 [55.6934182 -4.3555815]
 [55.6859453 -4.3414069]
 [55.6896599 -4.3292643]
 [55.6964595 -4.3525545]
 [55.7028635 -4.3566589]
 [55.7045988 -4.351543 ]
 [55.7028327 -4.3463191]
 [55.7004356 -4.3407223]
 [55.6941866 -4.3231996]
 [55.6913359 -4.3185243]
 [55.6890699 -4.3130225]
 [55.687036  -4.3079828]
 [55.6840153 -4.3021339]
 [55.6811411 -4.2990084]
 [55.6788471 -4.2912424]
 [55.6802265 -4.284701 ]
 [55.6621995 -4.2430467]
 [55.6572329 -4.2390082]
 [55.655983  -4.23264  ]
 [55.6552007 -4.2252601]
 [55.7072954 -4.3467533]
 [55.7052446 -4.3407187]
 [55.7030703 -4.3359135]
 [55.6976605 -4.3226198]
 [55.6959594 -4.3175033]
 [55.693707  -4.3109932]
 [55.6916351 -4.3072165]
 [55.6892417 -4.3020673]
 [55.686783  -4.2961061]
 [55.6824166 -4.2928225]
 [55.6834495 -4.2855856]
 [55.6811579 -4.278979 ]
 [55.6715497 -4.2544861]
 [55.6661212 -4.2424683]
 [55.6609738 -4.235968 ]
 [55.6599369 -4.2287444]
 [55.709604  -4.3405915]


In [3]:
with netCDF4.Dataset('Land/raw/C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.nc', 'r') as file:
    land = file.variables['lccs_class']
    lat = file.variables['lat']
    lon = file.variables['lon']

    # a list to store land cover classes occurred in current layout
    occur = []

    for i in range(turbine_loc.shape[0]):
        dist = np.abs(lat[:] - turbine_loc[i, 0])
        iy_min = dist.argmin()
        dist = np.abs(lon[:] - turbine_loc[i, 1])
        ix_min = dist.argmin()
        temp = land[0, iy_min, ix_min].data
        if temp not in occur:
            occur.append(int(temp))
        print(temp)
    
    print(occur)

180
180
180
180
70
180
180
180
180
180
180
180
180
110
180
70
110
110
70
100
70
110
110
180
180
180
210
180
180
70
70
70
70
180
180
110
100
70
70
180
180
180
130
180
180
180
70
180
180
100
70
100
180
180
130
130
180
180
180
110
100
180
180
100
70
100
100
130
130
130
130
180
180
110
70
110
110
180
100
70
70
70
100
70
70
110
180
70
70
110
180
180
70
70
70
70
110
180
110
100
70
180
100
130
100
70
70
70
100
100
110
70
70
100
70
70
70
70
100
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
100
100
70
100
70
70
70
70
70
70
110
70
70
110
70
70
70
70
70
70
70
70
70
70
70
70
70
110
100
110
70
100
100
70
70
70
70
180
70
70
70
180
180
100
130
70
70
70
100
100
100
100
70
70
110
70
70
70
70
70
100
130
100
70
70
180
180
70
110
70
70
70
100
70
70
110
110
70
70
70
70
[180, 70, 110, 100, 210, 130]


So, all the current wind turbines are installed in 6 main land cover categorises. They are:

70 Tree cover, needleleaved, evergreen, closed to open

100 Mosaic tree and shrub (>50%) / herbaceous cover (<50%) 

110 Mosaic herbaceous cover (>50%) / tree and shrub (<50%)

130 Grassland

180 Shrub or herbaceous cover, flooded, fresh/saline/brackish water

210 Water bodies.

These categories should be treated as feasible areas. Only 1 turbine is at the location of water bodies (210) due to the resolution issue, so 210 remains infeasible. Grassland (130) has already been feasible. So, Categories 70, 100, 110, and 180 should be changed from infeasible area to feasible area. Besides, categories which are similar to them should also be changed to feasible.

Therefore, change the following categories from infeasible to feasible:
40, 50, 60, 70, 80, 90, 100, 140, 160, 170, 180

## Get land cover classification
This part is for Yiyang to get the data.

In [4]:
def buffer(origin, lower, upper):
    """
    This function is to extend one element at the two ends of a list
    `origin`: an increasingly sorted numpy ndarray of inegers including the original data.
    `lower`: an integer of the lower bounds of the possible values.
    `upper`: an integer of the upper bounds of the possible values. 
    """
    origin = origin.tolist()
    if origin[0] > lower:
        origin.insert(0, origin[0] - 1)
    if origin[-1] < upper:
        origin.append(origin[-1] + 1)
    origin = np.array(origin, dtype='int32')
    return origin


area = [55.714704134580245, -4.364543821199962, 55.634359319706036, -4.183104393719847]
# This is the raw data of land cover classification
with netCDF4.Dataset('Land/raw/C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.nc', 'r') as file:
    land = file.variables['lccs_class']
    lat = file.variables['lat']
    lon = file.variables['lon']
    
    y_range = np.argwhere((lat[:] < area[0]) & (lat[:] > area[2])).T[0]
    x_range = np.argwhere((lon[:] < area[3]) & (lon[:] > area[1])).T[0]

    y_range = buffer(y_range, 0, lat.shape[0])
    x_range = buffer(x_range, 0, lon.shape[0])

    # flip the latitude
    y_range = np.flip(y_range, axis=0)

    print('The latitude grid is:\n', lat[y_range])
    print('The longitude grid is:\n', lon[x_range])

    land_cover = land[0, y_range, x_range].data
    print('The land cover classification is:\n', land_cover)


The latitude grid is:
 [55.63194444 55.63472222 55.6375     55.64027778 55.64305556 55.64583333
 55.64861111 55.65138889 55.65416667 55.65694444 55.65972222 55.6625
 55.66527778 55.66805556 55.67083333 55.67361111 55.67638889 55.67916667
 55.68194444 55.68472222 55.6875     55.69027778 55.69305556 55.69583333
 55.69861111 55.70138889 55.70416667 55.70694444 55.70972222 55.7125
 55.71527778]
The longitude grid is:
 [-4.36527778 -4.3625     -4.35972222 -4.35694444 -4.35416667 -4.35138889
 -4.34861111 -4.34583333 -4.34305556 -4.34027778 -4.3375     -4.33472222
 -4.33194444 -4.32916667 -4.32638889 -4.32361111 -4.32083333 -4.31805556
 -4.31527778 -4.3125     -4.30972222 -4.30694444 -4.30416667 -4.30138889
 -4.29861111 -4.29583333 -4.29305556 -4.29027778 -4.2875     -4.28472222
 -4.28194444 -4.27916667 -4.27638889 -4.27361111 -4.27083333 -4.26805556
 -4.26527778 -4.2625     -4.25972222 -4.25694444 -4.25416667 -4.25138889
 -4.24861111 -4.24583333 -4.24305556 -4.24027778 -4.2375     -4.2347222

In [5]:
# This is the feasibility data
with netCDF4.Dataset('Land/data/infeasible.nc', 'r') as file:
    fea = file.variables['feasible']
    lat = file.variables['latitude']
    lon = file.variables['longitude']
    
    y_range = np.argwhere((lat[:] < area[0]) & (lat[:] > area[2])).T[0]
    x_range = np.argwhere((lon[:] < area[3]) & (lon[:] > area[1])).T[0]

    y_range = buffer(y_range, 0, lat.shape[0])
    x_range = buffer(x_range, 0, lon.shape[0])

    feasibility = np.astype(np.logical_not(fea[y_range, x_range].mask), 'int32')
    print('The feasibility is:\n', feasibility)

The feasibility is:
 [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


## Performance of Current Layout

In [10]:
# get the grid cell indices of the current layout
turbine_ind = np.zeros(turbine_loc.shape[0], dtype='int32')
for i in range(turbine_ind.shape[0]):
    dist_sq = np.sum((conv.grid_gcs - turbine_loc[i]) ** 2, axis=1)
    turbine_ind[i] = dist_sq.argmin()  # This is the gene
turbine_ind = np.sort(turbine_ind)
print(turbine_ind)
print(conv.cols)

[  11   76  133  140  198  255  262  320  384  387  509  512  578  632
  634  637  642  649  695  707  729  755  764  767  772  775  786  788
  822  832  880  887  889  898  914  945  954  958  971  973 1010 1023
 1038 1068 1074 1077 1080 1098 1141 1145 1157 1195 1200 1209 1225 1246
 1249 1251 1254 1259 1267 1282 1284 1287 1326 1339 1341 1347 1375 1380
 1383 1386 1391 1406 1429 1432 1434 1439 1450 1463 1466 1470 1504 1509
 1515 1529 1553 1558 1561 1563 1569 1588 1596 1618 1637 1639 1647 1656
 1671 1682 1685 1690 1696 1703 1707 1713 1715 1721 1731 1750 1755 1760
 1763 1773 1781 1787 1796 1815 1819 1829 1833 1839 1841 1847 1855 1868
 1883 1893 1898 1914 1920 1926 1937 1941 1958 1965 1967 1971 1974 1979
 1986 1996 2002 2005 2018 2024 2031 2037 2039 2062 2066 2085 2089 2091
 2105 2130 2144 2148 2156 2158 2163 2165 2169 2173 2184 2188 2191 2209
 2216 2222 2233 2248 2251 2275 2282 2289 2298 2308 2317 2335 2341 2346
 2349 2373 2377 2401 2406 2423 2426 2436 2441 2473 2483 2486 2490 2494
 2501 

In [11]:
from Optimiser.config_ss import *

_wind_data = np.array([[2.3046875, 0.0390625],
                       [3.1015625, 0.125    ],
                       [2.8203125, 0.09375  ],
                       [2.5      , 0.078125 ],
                       [3.9375   , 0.140625 ],
                       [4.2890625, 0.2265625],
                       [3.9296875, 0.2265625],
                       [2.9140625, 0.078125 ]])



# The following part is from Optimiser.main

'''
xy position initialisation
from 1-D index to xy position
'''
rows = conv.rows
cols = conv.cols

xy = np.zeros((rows, cols, 2), dtype='float64')
for i in range(rows):
    xy[i, :, 1] = i
for i in range(cols):
    xy[:, i, 0] = i
xy = xy.reshape(rows * cols, 2)
xy = xy * cell_width + cell_width / 2
xy = xy.transpose()

# trans_matrix is for rotating the coordinates to fit different wind directions.
trans_matrix = np.zeros((len(theta) // 2, 2, 2), dtype='float64')
trans_xy = np.zeros((len(theta) // 2, rows * cols, 2), dtype='float64')
for i in range(len(theta) // 2):
    trans_matrix[i] = np.array(
        [[np.cos(theta[i]), -np.sin(theta[i])],
        [np.sin(theta[i]), np.cos(theta[i])]],
        dtype='float64')
    trans_xy[i] = np.matmul(trans_matrix[i], xy).transpose()

def fitness_func(ga_instance, solution, solution_idx):
    num_genes = ga_instance  # This line is changed
    fitness = 0  # a specific layout power accumulate
    for ind_t in range(len(theta) // 2):
        trans_xy_position = trans_xy[ind_t, solution, :]
        # print(trans_xy_position.shape)

        speed_deficiency0, speed_deficiency1 = wake(trans_xy_position, num_genes)

        # total power of a specific layout under a wind direction
        actual_velocity = (1 - speed_deficiency0) * _wind_data[ind_t, 0]
        lp_power = layout_power(actual_velocity, num_genes)
        fitness += lp_power.sum() * _wind_data[ind_t, 1]
        # calculation for the opposite wind direction
        actual_velocity = (1 - speed_deficiency1) * _wind_data[ind_t + 4, 0]
        lp_power = layout_power(actual_velocity, num_genes)
        fitness += lp_power.sum() * _wind_data[ind_t + 4, 1]

    # below is the part for calculating the effeciency
    num_genes = len(solution)
    wt_summary = np.zeros((num_genes,), dtype='float64')
    ideal_power = 0  # the ideal power of a wind turbine (kW)
    for ind_t in range(len(theta) // 2):
        trans_xy_position = trans_xy[ind_t, solution, :]

        speed_deficiency0, speed_deficiency1 = wake(trans_xy_position, num_genes)

        actual_velocity = (1 - speed_deficiency0) * _wind_data[ind_t, 0]
        lp_power = layout_power(actual_velocity, num_genes)  # total power of a specific layout specific wind speed specific theta
        wt_summary += lp_power * _wind_data[ind_t, 1]  # the weight of wind frequency at a given direction
        ideal_power += layout_power([_wind_data[ind_t, 0]], 1)[0] * _wind_data[ind_t, 1]

        actual_velocity = (1 - speed_deficiency1) * _wind_data[ind_t + 4, 0]
        lp_power = layout_power(actual_velocity, num_genes)  # total power of a specific layout specific wind speed specific theta
        wt_summary += lp_power * _wind_data[ind_t + 4, 1]  # the weight of wind frequency at a given direction
        ideal_power += layout_power([_wind_data[ind_t + 4, 0]], 1)[0] * _wind_data[ind_t + 4, 1]
    
    if ideal_power != 0:  # avoid the scenario of dividing zero
        wt_efficiency = wt_summary / ideal_power
    else:
        wt_efficiency = np.zeros((num_genes,), dtype='float64')
    efficiency = wt_efficiency.mean()
    print(f'fitness: {fitness}, efficiency: {efficiency}')
    fitness *= 24 * 365 * 56 / 1000
    household = fitness / 2.7
    print(f'power: {fitness}, household: {household}')


def wake(trans_xy_position, n):
    """
    This function is used by fitness_func().
    """
    # y value increasingly sort
    sorted_index = np.argsort(trans_xy_position[:, 1])
    wake_deficiency0 = np.zeros(n, dtype='float64')
    wake_deficiency1 = np.zeros(n, dtype='float64')
    for j in range(n):
        for k in range(j):
            dx = np.absolute(trans_xy_position[sorted_index[j], 0] - trans_xy_position[sorted_index[k], 0])
            dy = np.absolute(trans_xy_position[sorted_index[j], 1] - trans_xy_position[sorted_index[k], 1])
            d = cal_deficiency(dx=dx, dy=dy)
            # calculate the wake deficiency in two opposite directions at the same time.
            wake_deficiency0[sorted_index[k]] += d ** 2
            wake_deficiency1[sorted_index[j]] += d ** 2
    return np.sqrt(wake_deficiency0), np.sqrt(wake_deficiency1)


def cal_deficiency(dx, dy):
    """
    This function is used by wake().
    """
    r_wake = rotor_radius + entrainment_const * dy
    if dx >= rotor_radius + r_wake:
        intersection = 0
    elif dx > r_wake - rotor_radius:
        alpha = np.arccos((r_wake ** 2 + dx ** 2 - rotor_radius ** 2) / (2 * r_wake * dx))
        beta = np.arccos((rotor_radius ** 2 + dx ** 2 - r_wake ** 2) / (2 * rotor_radius * dx))
        intersection = alpha * r_wake ** 2 + beta * rotor_radius ** 2 - r_wake * dx * np.sin(alpha)
    else:
        intersection = np.pi * rotor_radius ** 2
    return 2.0 / 3.0 * intersection / (np.pi * r_wake ** 2)


def layout_power(v, n):
    """
    This function is used by fitness_func().
    Power unit: kW
    """
    power = np.zeros(n, dtype='float64')
    for j in range(n):
        if 2.0 <= v[j] < 18:
            if v[j] < 12.8:
                power[j] = 0.3 * v[j] ** 3
            else:
                power[j] = 629.1
    return power

fitness_func(turbine_ind.shape[0], turbine_ind, None)

fitness: 1922.5936863644342, efficiency: 0.6000612825348957
power: 943147.5587829369, household: 349313.91066034697


In [8]:
# Display the current layout
import folium.features
import folium.raster_layers
import folium

m = folium.Map(location=[55.674099775230026, -4.271278381347657], zoom_start=12)

image_file = 'https://github.com/ShitianZhang22/Wind-Farm-Layout-Optimisation/blob/main/icon/turbine.png?raw=true'

for i in range(turbine_loc.shape[0]):
    folium.Marker(
        location=[turbine_loc[i, 0], turbine_loc[i, 1]],
        icon=folium.CustomIcon(icon_image=image_file, icon_size=(20, 20), icon_anchor=(10, 10)),
        color='red'
    ).add_to(m)

folium.Rectangle([[55.634359319706036, -4.364543821199962], [55.714704134580245, -4.183104393719847]]).add_to(m)
m

## Optimisation
Here we try to show the optimised layout.

In [12]:
optimised_layout =  [1273, 1386, 2756, 2285, 936, 2357, 221, 294, 1616, 1667, 2040, 807, 2583, 487, 292, 2389, 89, 1548, 826, 1250, 680, 2514, 1001, 2448, 2778, 431, 2643, 1971, 2800, 197, 929, 1035, 1116, 2250, 1774, 387, 1454, 1699, 2261, 2150, 2372, 2806, 890, 2304, 583, 1993, 21, 1209, 1819, 2768, 1582, 0, 1304, 1574, 2176, 2570, 185, 452, 1193, 2413, 2789, 1859, 118, 968, 412, 2553, 1557, 2478, 1639, 2649, 93, 1425, 2482, 2736, 29, 13, 519, 1890, 342, 2237, 1934, 1752, 1906, 738, 419, 2330, 1922, 611, 1297, 1011, 2527, 2790, 754, 1468, 106, 1790, 2675, 600, 327, 1143, 1644, 2601, 262, 364, 2214, 1921, 2500, 2725, 2019, 250, 551, 1981, 1240, 2747, 2636, 533, 2315, 2605, 2494, 1483, 130, 942, 1327, 1632, 1162, 624, 802, 682, 114, 1444, 1813, 567, 39, 770, 77, 902, 503, 132, 1879, 787, 2405, 914, 1353, 2728, 442, 2816, 2282, 2190, 2125, 237, 2094, 2455, 2837, 285, 855, 874, 1232, 2470, 58, 1177, 2104, 243, 2826, 1829, 1734, 1535, 2658, 1737, 830, 1123, 395, 2049, 2593, 2701, 2059, 648, 2418, 1369, 98, 2109, 1093, 1221, 312, 1056, 1105, 712, 2688, 1509, 227, 1864, 2069, 2225, 1018, 1394, 171, 984, 2669, 727, 636, 157, 2710, 65, 1181, 1499, 1839, 469, 1680, 7, 668, 2618, 352, 1600, 214, 146, 2843]
optimised_layout.sort()
optimised_layout

[0,
 7,
 13,
 21,
 29,
 39,
 58,
 65,
 77,
 89,
 93,
 98,
 106,
 114,
 118,
 130,
 132,
 146,
 157,
 171,
 185,
 197,
 214,
 221,
 227,
 237,
 243,
 250,
 262,
 285,
 292,
 294,
 312,
 327,
 342,
 352,
 364,
 387,
 395,
 412,
 419,
 431,
 442,
 452,
 469,
 487,
 503,
 519,
 533,
 551,
 567,
 583,
 600,
 611,
 624,
 636,
 648,
 668,
 680,
 682,
 712,
 727,
 738,
 754,
 770,
 787,
 802,
 807,
 826,
 830,
 855,
 874,
 890,
 902,
 914,
 929,
 936,
 942,
 968,
 984,
 1001,
 1011,
 1018,
 1035,
 1056,
 1093,
 1105,
 1116,
 1123,
 1143,
 1162,
 1177,
 1181,
 1193,
 1209,
 1221,
 1232,
 1240,
 1250,
 1273,
 1297,
 1304,
 1327,
 1353,
 1369,
 1386,
 1394,
 1425,
 1444,
 1454,
 1468,
 1483,
 1499,
 1509,
 1535,
 1548,
 1557,
 1574,
 1582,
 1600,
 1616,
 1632,
 1639,
 1644,
 1667,
 1680,
 1699,
 1734,
 1737,
 1752,
 1774,
 1790,
 1813,
 1819,
 1829,
 1839,
 1859,
 1864,
 1879,
 1890,
 1906,
 1921,
 1922,
 1934,
 1971,
 1981,
 1993,
 2019,
 2040,
 2049,
 2059,
 2069,
 2094,
 2104,
 2109,
 2125,
 2

In [19]:
m = folium.Map(location=[55.674099775230026, -4.271278381347657], zoom_start=12)

image_file = 'https://github.com/ShitianZhang22/Wind-Farm-Layout-Optimisation/blob/main/icon/turbine.png?raw=true'

optimised_loc = conv.gene_to_pos(optimised_layout)

for i in range(len(optimised_loc)):
    folium.Marker(
        location=[optimised_loc[i][0], optimised_loc[i][1]],
        icon=folium.CustomIcon(icon_image=image_file, icon_size=(20, 20), icon_anchor=(10, 10)),
        color='red'
    ).add_to(m)

folium.Rectangle([[55.634359319706036, -4.364543821199962], [55.714704134580245, -4.183104393719847]]).add_to(m)
m